# CSCI 4181/6802: Tutorial 2
## Support Vector Machines for Protein Classification

Version: 1.3 (February 3, 2020)

This is the second of four tutorials in the course that will give you practical experience with biological data. The exercises can be completed during class or later this week. The due date for this assignment is **Monday, February 10, 2020**, before 11:59pm. Please email your tutorial to me at **rbeiko@dal.ca**. Questions that you need to answer are indicated in **boldface**. Each question is worth one point unless otherwise noted.

The goal of this tutorial is to apply support vector machines to the characterization of proteins that contain membrane-spanning domains. These domains tend to be hydrophobic, enriched in certain types of amino acids (A, E, K, L, M), and deficient in others (D, G, P). We will try out the scikit-learn interface to the popular *libsvm* package to try and classify our sequences. Ultimately, our goal is to pick out true alpha-helix transmembrane sequences from a background of "null", or negative, data that is generated by shuffling the order of the amino acids in the alpha-helix transmembrane protein set.

### Software Pre-requisites

If you were able to run Tutorial 1, you should have no issues with Tutorial 2. No additional software is required.

In these tutorials, whenever you see a code block, you must enter it by clicking on it and execute it by hitting Shift+Enter. This will change the [ ] icon to the left to [\*], indicating that the code is running. It will indicate a number once the code has finished executing. **Be sure to run every code block in order, as they depend on one another.**

The next code block loads all of the required modules. If you have any installation issues, the following code block will generate an error. Make sure to run this first and e-mail rbeiko@dal.ca if you are having any issues.

In [1]:
# If you can run this code block without errors, you have everything you need!
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd

### Data Pre-requisites

The dataset we will use was downloaded from the PDBTM database, a subset of the Protein Data Bank (PDB) dedicated
to transmembrane proteins. The page with URL http://pdbtm.enzim.hu/data/pdbtm_alpha.seq hosts a long list of
protein sequences that are known to contain transmembrane domains; it is from this set that our positive cases were
derived. Sequences with the same prefix are presumably homologous, so I chose to extract only the longest sequence
from each homologous set. This accomplished three worthwhile goals:

- (i) Reducing the size of the dataset
- (ii) Eliminating short sequences with potentially uninformative compositional vectors
- (iii) Reducing bias in the training set by including no more than one homolog from any given set

Further to (ii) above, I also used a minimum length criterion to filter the sequences: to be included, a given sequence
must be at least 100 amino acids long. These criteria reduced the size of our positive set from >2800 to 599 sequences.

Let's take a peek at the data file:

In [2]:
# Print out the first two lines of the file, just for a peek
with open("tutorial2_appendix_data/AlphaTM.faa", 'r') as alpha_tm:
    print(alpha_tm.readline(), end='')
    print(alpha_tm.readline(), end='')
    print(alpha_tm.readline(), end='')
    print(alpha_tm.readline(), end='')

>2a06_P
MTNIRKSHPLMKIVNNAFIDLPAPSNISSWWNFGSLLGICLILQILTGLFLAMHYTSDTTTAFSSVTHICRDVNYGWIIRYMHANGASMFFICLYMHVGRGLYYGSYTFLETWNIGVILLLTVMATAFMGYVLPWGQMSFWGATVITNLLSAIPYIGTNLVEWIWGGFSVDKATLTRFFAFHFILPFIIMAIAMVHLLFLHETGSNNPTGISSDVDKIPFHPYYTIKDILGALLLILALMLLVLFAPDLLGDPDNYTPANPLNTPPHIKPEWYFLFAYAILRSIPNKLGGVLALAFSILILALIPLLHTSKQRSMMFRPLSQCLFWALVADLLTLTWIGGQPVEHPYITIGQLASVLYFLLILVLMPTAGTIENKLLKW
>2a0d_A
MESPIQIFRGEPGPTCAPSACLPPNSSAWFPGWAEPDSNGSAGSEDAQLEPAHISPAIPVIITAVYSVVFVVGLVGNSLVMFVIIRYTKMKTATNIYIFNLALADALVTTTMPFQSTVYLMNSWPFGDVLCKIVISIDYYNMFTSIFTLTMMSVDRYIAVCHPVKALDFRTPLKAKIINICIWLLSSSVGISAIVLGGTKVREDVDVIECSLQFPDDDYSWWDLFMKICVFIFAFVIPVLIIIVCYTLMILRLKSVRLLSGSREKDRNLRRITRLVLVVVAVFVVCWTPIHIFILVEALGSTSHSTAALSSYYFCIALGYTNSSLNPILYAFLDENFKRCFRDFCFPLKMRMERQSTSRVRNTVQDPAYLRDIDGMNKPV


Our data file is in the text-based FASTA format file with extension *.faa* indicating that the sequences are amino acids, rather than nucleotides. There is one line for the label, and one line for the protein sequence. Our goal is to identify these postiviely as transmembrane proteins against a background of negative data. The sequence file is small (255KB) so let's read it into memory for ease of use:

In [3]:
def get_positive_sequences():
    # Dictionary with protein ID as the key and the sequence as the value
    positive_sequences = {}

    with open("tutorial2_appendix_data/AlphaTM.faa", 'r') as alpha_tm:
        for label in alpha_tm:
            # Cut out the FASTA label marker character > by excluding the first character
            # Get rid of any newline characters with .strip()
            label = label[1:].strip()
            # Get the sequence on the next line, remove newline characters with .strip()
            sequence = alpha_tm.readline().strip()
            positive_sequences[label] = sequence
    return positive_sequences

### Negative Set

The negative set for today's event will actually be generated directly from the positive set. This is in contrast to the last tutorial where we were using data from two organisms. Here we will generate a "null distribution" of sequences by taking each positive sequence, splitting it into fragments of length L, and randomly reassembling these fragments to generate a new sequence. So if L = 1, then we are shuffling the order of amino acids. But if L = 10, then long contiguous stretches of the protein are going to be preserved. We are breaking the sequences into adjacent words of length L, shuffling those words randomly, and then merging them together. This is a sampling without replacement strategy.

In [4]:
from random import shuffle

def shuffle_sequences(positive_sequences, L = 1):
    negative_sequences = {}
    # Iterate through the key, value pairs of the positive_sequences dictionary
    for label, positive_sequence in positive_sequences.items():
        # Break the original into words, e.g., "ABCDEFG" becomes ["AB" "CD" "EF" "G"]
        words = [ positive_sequence[i:i+L] for i in range(0, len(positive_sequence), L) ]
        # Shuffles the words array in place, e.g., ["CD", "G", "EF", "AB"]
        shuffle(words)
        # Merge the shuffled words back together, e.g., "CDGEFAB"
        negative_sequence = "".join(words)
        # Store in the dictionary
        negative_sequences[label] = negative_sequence
        # This will raise an error if our negative sequences aren't the length we expect
        assert(len(positive_sequence) == len(negative_sequence))
    return negative_sequences

### Selecting an Alphabet

We can reduce the 20-letter amino acid alphabet by mapping the characters onto a smaller set. This can be done using empirical properties of the amino acids. Three mappings have been provided here. The first is simply the identity, which uses all of the amino acids. The second is the PAM 6-letter alphabet, which categorizes amino acids based on chemical properties granted by their side chains. The third is a 3-letter alphabet that maps amino acids as either positively charged, aromatic, or proline. Later in the tutorial, you will be asked to change the alphabet used and investigate the impact that a reduced alphabet has on the accuracy scores.

In [5]:
# This data set has 21 amino acid codes (20 actual amino acids + X for wildcard)
amino_acids = "ACDEFGHIKLMNPQRSTVWXY"
# Use the standard 20 amino acid alphabet plus X for wildcard
amino_acid_alphabet = "ACDEFGHIKLMNPQRSTVWXY"

# Map each of the 20 amino acids into 6 categories, encoded by A, B, C, D, E, and F (plus X) 
PAM6_mapping = {"D":"A", "E":"A", "N":"A", "Q":"A",  # Acidic and amide amino acids
                "W":"B", "F":"B", "Y":"B", # Aromatic amino acids
                "H":"C", "K":"C", "R":"C", # Basic amino acids
                "A":"D", "G":"D", "I":"D", "L":"D", "V":"D", # Aliphatic amino acids
                "C":"E", "M":"E", "S":"E", "T":"E", # Hydroxyl or sulphur amino acids
                "P":"F", # Proline, all alone
                "X":"X"} # Maintain wildcard amino acids as X
# Put it into a string the same length as amino_acids, in corresponding order
PAM6_alphabet = "".join( [ PAM6_mapping[ amino_acids[i] ] for i in range(0, len(amino_acids)) ] )

# Map each of the 20 amino acids into 3 categories, encoded by A, B, and C (plus X)
three_mapping = {"M":"A", "A":"A", "L":"A", "E":"A", "K":"A", # Positive
                  "C":"B", "D":"B", "F":"B", "G":"B", "H":"B", "I":"B", "N":"B", # Aromatic
                  "Q":"B", "R":"B", "S":"B", "T":"B", "V":"B", "W":"B", "Y":"B", # Aromatic continued
                  "P":"C", # Proline
                  "X":"X"} # Maintain wildcard amino acids as X
# Put it into a string the same length as amino_acids, in corresponding order
three_alphabet = "".join( [ three_mapping[ amino_acids[i] ] for i in range(0, len(amino_acids)) ] )

# You can design your own alphabet mapping by creating a string that maps amino_acids onto the new characters
# For example,
awful_alphabet = "AAAAAAAAAABBBBBBBBBBB"
# This alphabet will convert the amino acids ACDEFGHIKL to A and the amino acids MNPQRSTVWXY to B
# Try your own, and select it in the next code block!

### Building the Protein Features

As in Tutorial 1, we need to derive tractable features from our protein sequences. We will again count the amino acid words of length w. For example, the sequence "MSIVLKVKVK" with w = 3 would have the features and counts: MSI: 1, SIV: 1, IVL: 1, VLK: 1, LKV: 1, KVK: 2, 

In [6]:
# We use islice to more efficiently count our words
# This is in the standard Python library, so everyone should have this
from itertools import islice

# This function efficiently slides through a sequence, returning a list of words of length w
def window(seq, w = 1):
    "Returns a sliding window (of width w) over data from the iterable"
    "   s -> (s0,s1,...s[w-1]), (s1,s2,...,sw), ...                   "
    it = iter(seq)
    result = tuple(islice(it, w))
    if len(result) == w:
        yield "".join(result)
    for elem in it:
        result = result[1:] + (elem,)
        yield "".join(result)

# This function iterates through the sequences, getting the word counts for each sequence
def count_words(sequence_dictionary, w = 1):
    # Store each protein's count dictionary in another dictionary
    sample_dict = {}
    for label, sequence in sequence_dictionary.items():
        # Translate the sequence based on our dictionary ro reduce the alphabet
        translated_sequence = sequence.translate(sequence.maketrans(amino_acids, my_alphabet))
        # Use the above window function to get a list of words, e.g., ["AA", "AC", "CA", "AC", ...]
        words = list(window(translated_sequence, w))
        # Use np.unique to count the unique words and get two lists e.g.:
        # unique = ["AA", "AC, "CA", ...]
        # counts = [1, 2, 1, ...]
        unique, counts = np.unique(words, return_counts=True)
        # Transform the two lists into a dictionary, e.g., {"AA": 1, "AC": 2, "CA": 1, ...}
        feature_dict = dict((word, count) for word, count in zip(unique,counts))
        # Put it as an entry in the sample_dict so we have one feature dict per protein sequence
        sample_dict[label] = feature_dict
    return sample_dict

Finally, we need a function that lets us invoke a Support Vector Machine-based classifier. This time, we will be using cross-validation to score the accuracy. If you'd like to learn more about the parameters C and gammma for the RBF kernel, see http://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html.

In [7]:
def SVM(X, y, C = 1, gamma = 'auto', cv = 5):
    classifier = SVC(kernel='rbf', C = C, gamma = gamma)
    scores = cross_val_score(classifier, X, y, cv = cv)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

The code block below is the workhorse that calls all of our established code above. Make sure you've run all of the code blocks above this point. Once they have been run, only the code below needs to be re-run once the parameters have been changed.

In [8]:
########### BEGIN PARAMETERS THAT CAN BE MODIFIED ###########

# Negative/null data creation parameter
L = 1 # *** Shuffle parameter, defines the contiguous sequence lengths to shuffle for negative data ***

# Feature creation parameter
w = 1 # *** Word size parameter, defines the length of the words to use for the features ***

# Alphabet mapping parameter
my_alphabet = amino_acid_alphabet # *** Alphabet mapping, options: amino_acid_alphabet, PAM6_alphabet, three_alphabet

# SVM parameters
C = 1 # SVM penalty parameter
gamma = 'auto' # SVM RBF kernel coefficient, 'auto' defaults to 1/n_features, but can be set to any float

# Cross-validation parameters
cv = 5 # Number of cross-validation folds to perform

########### END PARAMETERS THAT CAN BE MODIFIED ###########

# Get the positive sequences from the data file, using the function described above
positive_sequences = get_positive_sequences()

# Count the words in our positive_sequences {label: sequence} dictionary
positive_sample_dict = count_words(positive_sequences, w = w)

# Convert to Pandas DataFrame for easy manipulation
positive_matrix = pd.DataFrame.from_dict(positive_sample_dict, orient='index').fillna(0)

# Here we call our shuffle_sequences. You will be asked to modify the second parameter, word length
negative_sequences = shuffle_sequences(positive_sequences, L = L)

# Count the words in our negative_sequences {label: sequence} dictionary
negative_sample_dict = count_words(negative_sequences, w = w)

# Convert to Pandas DataFrame for easy manipulation
negative_matrix = pd.DataFrame.from_dict(negative_sample_dict, orient='index').fillna(0)

# Create the label vector, 1 = positive data, 0 = negative data
labels = [1]*positive_matrix.shape[0] + [0]*negative_matrix.shape[0]

# Combine our two genome count tables into one table, making sure to fill NAs with 0
# NAs can occur when we append matrices with different sets of words (i.e., a word was observed in one genome)
# but not the other)
combined_matrix = positive_matrix.append(negative_matrix).fillna(0)

# Call the classifier on our matrix and genome labels
SVM(combined_matrix, labels, C = C, gamma = gamma, cv = cv)

Accuracy: 0.50 (+/- 0.00)


## Tutorial Questions

**Q1: Given the algorithm that was used to create the negative data set, does it make sense to set w = 1? Why or why not?**

_Response for Q1_

For the rest of this tutorial, choose a new integer value for `w` that is greater than 1. Remember that larger values will generate more features, which requires more RAM and time, so do not set this too high.

We can change the alphabet to a smaller one by setting `my_alphabet` to `PAM6_alphabet` or `three_alphabet`.

**Q2: What are the potential advantages and disadvantages of recoding an alphabet of size k down to a
smaller size (i.e., fewer characters)?**

_Response for Q2_

Let's try the algorithm with the standard alphabet, and then the reduced alphabets. You will use the 6-category and 3-category one that I have provided, and then make up one of your own. Change the `my_alphabet` parameter to either `PAM6_alphabet` or `three_alphabet`. Create your own alphabet based on some properties or groupings of amino acids, or just make a random one if you're not so biologically inclined (though try something more interesting than I did with `awful_alphabet`, but use that as a formatting guide if you need it).

**Q3: Before running the classifier, make note of the L and w parameters and be sure not to change them when comparing the effect of the alphabet size. Rerun the classifier and record your new accuracy scores with each of the different alphabets. Describe your custom alphabet and show the CV accuracy below. "Standard" means the default 20 amino acid alphabet with no reduction.**

_Response for Q3_

| Alphabet |  CV Accuracy  | 
| :------- | ------------- |
| Standard |               |
|   PAM6   |               |
|   Three  |               |
|  Custom  |               |

In the SVM function we defined, you will notice that we use the option `kernel='rbf'`. This means that we are training the Support Vector Machine classifier using a radial basis function kernel.

**Q4: What is the purpose of using a radial basis function kernel for SVM classification?**

_Response for Q4_

We have chosen 5 as the number of folds for cross-validation above. As you increase the number of folds, eventually you will hit the special case known as leave-one-out cross validation once the number of folds hits the number of features. In this case, only one data point is withheld, with the classifier being trained on the rest. scikit-learn has a LeaveOneOut() function that can be used for this purpose (http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html).

**Q5: You don’t need to run this (it will take a while!) but would you expect 5-fold or leave-one-out cross validation
to give higher accuracy scores? Why?**

_Response for Q5_

You have tried the default SVM parameters, but it turns out that two parameters can exert a large influence on the final accuracy of the SVM: these are C and γ (gamma). C is the error penalty; higher values of C will induce heavier penalties on misclassified points, but can lead to overfitting (and consequently bad generalization). Gamma modifies the shape of the kernel function while retaining the same degree.

One useful option offered by scikit-learn is the ability to do a grid search, which explores a range of combinations of C and γ settings. See http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html for details on the grid search implementation, if interested. We will be doing a quicker and simpler scan of the SVM parameter values.

**Q6: Try searching for an optimal value of C by running the SVM classifier with at least ten different values for C. The authors of the original LIBSVM tool recommend trying `C = 2k`, for a wide range of k values. Record the relationship between k (i.e., C/2) and your cross-validation accuracy. Fill in the table below, with the first accuracy value resulting from C=1.**

_Response for Q6_

|    k     |  CV Accuracy  | 
| :------- | ------------- |
|   0.5    |               |
|          |               |
|          |               |
|          |               |
|          |               |
|          |               |
|          |               |
|          |               |
|          |               |
|          |               |

**Q7 [2 points]: In this tutorial we have focused on converting sequences into feature vectors, possibly with
recoding; the SVM then takes the dot product of these vectors during training. We have used the radial basis function kernel, but any function that computes the similarity between a pair of sequences could be used. Can you suggest a different kernel function that might be useful in distinguishing alpha-helix transmembrane proteins from their randomized equivalents? You do not have to implement this function, but provide a description of how the similarity could be measured, and a brief explanation of why this might be an improvement over the standard kernels.**

_Response for Q7_

By now, you should have a pretty good sense of which strategies worked well and which worked less well. You might
also want to try changing the kernel, since RBF is not always the best choice! Other options are 'linear', 'poly', and 'sigmoid'. See http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC for complete details.

**Q8 [2 points]: Play around with any of the parameters (but focus on C, gamma, L, w). Accuracy scores of 90% are easy to attain. Can you do better? Describe the parameters that you varied and the ranges of values that you tried. Report your highest score and each of the parameters involved in getting that score.**

_Response for Q8_